In [30]:
# origin dataset https://www.kaggle.com/a2015003713/militaryaircraftdetectiondataset
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw
import cv2

In [31]:
class TaskSpecificDataset():
    
    def __init__(self, root: str):
        self.root = root
        self.data = []
        self.features = []
        self.images = []
        self.__traverse_folder__(self.root)
        self.__filter_dataset__()
    
    def get_features(self):
        return self.features
    
    def __traverse_folder__(self, path: str):
        for entry in os.scandir(path):
            if entry.is_file():
                self.data.append(entry.name)
    
    def __filter_dataset__(self):
        self.features = []
        self.images = []
        
        for item in self.data:
            if item.__contains__(".csv"):
                self.features.append(item)
            elif item.__contains__(".jpg"):
                self.images.append(item)

In [32]:
root = "input/archive_jets/dataset"
dataset = TaskSpecificDataset(root)
result = dataset.get_features()
print(len(result))

3033


In [33]:
df = pd.DataFrame(data={})
for item in result: 
    new_df = pd.read_csv(os.path.join(root, item))
    df = df.append(new_df)
    
df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,0041e69431bf872309d1aff628b6494f,1279,725,E2,217,281,1198,616
0,0079ecb92d4a99dc405dae5bde86d8a9,1200,799,F15,260,163,1002,618
0,0081dca65c2991b4c145de6af9845946,1000,667,B2,50,418,952,519
0,00b2add164cb42440a52064e390ea3d2,1280,850,B1,322,112,893,618
0,00b627cb5578bb036edb01cdcf7b56d9,1300,867,F35,607,204,1202,529
...,...,...,...,...,...,...,...,...
0,ff9debe70ce7afb8ef2cb1f9d8edd44d,2479,1983,B1,1375,542,1926,1141
0,ffa565e4f573979207ea525e63c204d5,3000,2209,F18,399,1311,1209,1882
1,ffa565e4f573979207ea525e63c204d5,3000,2209,F18,1840,480,2690,1083
0,ffa8d739b58bbf6b65e6d5dd9b2e8a69,1800,1177,B1,108,417,1720,738


In [34]:
X_train, y_train, X_test, y_test = train_test_split(df.drop(["class"], axis=1), df["class"], test_size=0.25)

In [35]:
name = ""
box = [0, 0, 0, 0]
for idx, data in df.iterrows():
    name = data["filename"]
    box[0] = data["xmin"]
    box[1] = data["ymin"]
    box[2] = data["xmax"]
    box[3] = data["ymax"]
    break
#     print(i)
#     print(j["filename"])
# img = Image.open(os.path.join(root, name + ".jpg"))
mat = cv2.imread(os.path.join(root, name + ".jpg"))
height, width, channel = mat.shape[:3]
size = mat.size
print(mat.size)
print(mat.shape)
img = mat

2781825
(725, 1279, 3)


array([[242, 200, 165],
       [242, 200, 165],
       [242, 200, 165],
       ...,
       [162, 174, 176],
       [162, 174, 176],
       [162, 174, 176]], dtype=uint8)

In [48]:
new_mat = mat.reshape( -1)
print(len(new_mat))
# new_mat[::1]
new_mat

2781825


array([242, 200, 165, ..., 162, 174, 176], dtype=uint8)

In [8]:


def draw_box(image, box):
#     draw = ImageDraw.Draw(image)
#     draw.rectangle(box, fill ="#000000", outline=(0,0,0))
    color = (255, 255, 255)
    cv2.rectangle(
            image,
            box,
#             (int(box[0]), int(box[1])),
#             (int(box[2]), int(box[3])),
            color, 2
        )
    cv2.putText(image, "classes[i]", (int(box[0]), int(box[1]-5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
                    lineType=cv2.LINE_AA)
    return image

In [9]:
def drawBoundingBoxes(imageData, imageOutputPath, inferenceResults, color):
    """Draw bounding boxes on an image.
    imageData: image data in numpy array format
    imageOutputPath: output image file path
    inferenceResults: inference results array off object (l,t,w,h)
    colorMap: Bounding box color candidates, list of RGB tuples.
    """
    for res in inferenceResults:
        left = int(res['left'])
        top = int(res['top'])
        right = int(res['right'])#int(res['left']) + int(res['width'])
        bottom = int(res['bottom'])#int(res['top']) + int(res['height'])
        label = res['label']
        imgHeight, imgWidth, _ = imageData.shape
        thick = int((imgHeight + imgWidth) // 900)
#         print left, top, right, bottom
        cv2.rectangle(imageData,(left, top), (right, bottom), color, thick)
        cv2.putText(imageData, label, (left, int(top - 12)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, thickness = int(thick // 3), lineType=cv2.LINE_AA)
        
    cv2.imwrite(os.path.join(root, "!test.jpg"), imageData)

    return imageData

In [10]:
results = [
        {
            "left": box[0],
            "top": box[1],
            "right": box[2],
            "bottom": box[3],
            "label": "label"
        }
]
image = drawBoundingBoxes(img, "", results, (255, 255, 255))
image

# image = draw_box(img, box)
# image

array([[[242, 200, 165],
        [242, 200, 165],
        [242, 200, 165],
        ...,
        [222, 177, 134],
        [222, 177, 134],
        [221, 176, 133]],

       [[242, 200, 165],
        [242, 200, 165],
        [242, 200, 165],
        ...,
        [222, 177, 134],
        [221, 176, 133],
        [221, 176, 133]],

       [[242, 200, 165],
        [242, 200, 165],
        [242, 200, 165],
        ...,
        [222, 177, 134],
        [221, 176, 133],
        [220, 175, 132]],

       ...,

       [[165, 177, 179],
        [166, 178, 180],
        [167, 179, 181],
        ...,
        [159, 171, 173],
        [159, 171, 173],
        [159, 171, 173]],

       [[165, 177, 179],
        [166, 178, 180],
        [167, 179, 181],
        ...,
        [160, 172, 174],
        [160, 172, 174],
        [160, 172, 174]],

       [[167, 179, 181],
        [168, 180, 182],
        [169, 181, 183],
        ...,
        [162, 174, 176],
        [162, 174, 176],
        [162, 174, 176]]